# Programming Assignment

This programming assignment is migrated from tensorflow 2.0 exercise, aiming at familiar with pytorch programming API

Reference:
* https://nextjournal.com/gkoehler/pytorch-mnist
* https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
* https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html
* https://blog.csdn.net/touristourist/article/details/100535544
* Transform - https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

## Model validation on the Iris dataset

#### The Iris dataset

In this assignment, you will use the [Iris dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html). It consists of 50 samples from each of three species of Iris (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimeters. For a reference, see the following papers:

- R. A. Fisher. "The use of multiple measurements in taxonomic problems". Annals of Eugenics. 7 (2): 179–188, 1936.

Your goal is to construct a neural network that classifies each sample into the correct class, as well as applying validation and regularisation techniques.

In [1]:
import numpy as np
import torch
import torchvision
from torch import nn
from torch.nn.functional import one_hot
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

#### Load and preprocess the data

In [2]:
from typing import Optional, Callable

class IrisDataSet(Dataset):
    """ Iris dataset load from sklearn
    """
    def __init__(self, train_test_split: float=0.15, transform: Optional[Callable]=None):
        
    

In [3]:
MNIST_train = datasets.MNIST('./data/minst/', train=True, download=True, transform=trans)
MNIST_train.data.shape

torch.Size([60000, 28, 28])

In [5]:
MNIST_test = datasets.MNIST('./data/minst/', train=False, download=True, transform=trans)
MNIST_test.data.shape

torch.Size([10000, 28, 28])

In [6]:
batch_size_train = 1000
batch_size_test = 1000

# 1, torchvision.transforms.ToTensor(): swap color axis because
# numpy image: H x W x C
# torch image: C x H x W
train_loader = DataLoader(
    datasets.MNIST('./data/minst/', train=True, download=True, transform=trans),
    batch_size=batch_size_train, 
    shuffle=True
)
test_loader = DataLoader(
    datasets.MNIST('./data/minst/', train=False, download=True, transform=trans),
    batch_size=batch_size_test, 
    shuffle=True
)

In [7]:
class SequentialNetwork(nn.Module):
    def __init__(self):
        super(SequentialNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.LazyConv2d(out_channels=8, kernel_size=(3,3), padding='same'),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            nn.Flatten(),
            nn.LazyLinear(64),
            nn.ReLU(),
            nn.LazyLinear(64),
            nn.ReLU(),
            nn.LazyLinear(10),
            nn.Softmax(dim=1),
        )
    
    def forward(self, x):
        return self.linear_relu_stack(x)

In [8]:
import torch.optim as optim

model = SequentialNetwork()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

/Users/i058959/miniconda3/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [9]:
from torch.nn.functional import one_hot

def sparse_cross_entropy_loss(y_pred, y_true):
    return torch.mean(-torch.log(torch.sum(one_hot(y_true, num_classes=len(y_pred[0])) * y_pred, axis=1)))

In [10]:
def train():
    model.train()
    size = len(train_loader.dataset)
    total_batch = len(train_loader)
    for epoch in range(10):
        accuracy = 0
        for batch, (X, y) in enumerate(train_loader):
            # Compute prediction and loss
            pred = model(X)
            loss = sparse_cross_entropy_loss(pred, y)
            accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()

            # Backpropgation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch == total_batch - 1:
                loss, accuracy = loss.item(), accuracy/size
                print(f'Epoch {epoch} - loss: {loss:>7f}, accuracy: {accuracy:>7f}')

In [11]:
def test():
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in test_loader:
            pred = model(X)
            test_loss += sparse_cross_entropy_loss(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
train()

Epoch 0 - loss: 0.528036, accuracy: 0.684117
Epoch 1 - loss: 0.320412, accuracy: 0.889800
Epoch 2 - loss: 0.250412, accuracy: 0.924000
Epoch 3 - loss: 0.196382, accuracy: 0.937767
Epoch 4 - loss: 0.174585, accuracy: 0.948417
Epoch 5 - loss: 0.122036, accuracy: 0.955533
Epoch 6 - loss: 0.123904, accuracy: 0.959683
Epoch 7 - loss: 0.086862, accuracy: 0.963883
Epoch 8 - loss: 0.095903, accuracy: 0.968117
Epoch 9 - loss: 0.097329, accuracy: 0.969900


In [13]:
test()

Test Error: 
 Accuracy: 96.8%, Avg loss: 0.103535 

